In [1]:
from tkinter import *
import tkinter as tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import ImageTk
from PIL import Image, ImageTk
import tkinter.ttk as ttk
from datetime import datetime
from datetime import timedelta
from pytz import timezone

In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


'C:\Users\minyoung'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [4]:
df= pd.read_csv('merged_final.csv')

# 머신러닝 코드

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import math

data = pd.read_csv("merged_final.csv")

#Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le = le.fit(data['area'])
data['area2'] = le.transform(data['area'])
data['area2'] = data['area2'].astype('category')

data['day'] = data['date'].str.split('-').str[2]
data['month'] = data['date'].str.split('-').str[1]
data['year'] = data['date'].str.split('-').str[0]

data.drop(['area', 'date'], axis = 1, inplace = True)

#발전량 이상치 제거 & 정규화
data = data[data['generation'] != 0]

#이상치라고 생각되는 값만 빼고 추출
data = data[(data['generation'] < 3000) & (data['generation'] >1)]
data = data[data['avgWs'] < 6]
data = data[data['PM10'] < 150]
data = data[data['PM25'] < 80]

data['day'] = data['day'].astype(float)
data['year'] = data['year'].astype(float)
data['month'] = data['month'].astype(float)
data['area2'] = data['area2'].astype(float)
data = data.reset_index(drop = True)

#Encoding에 앞서 변수 분리
y_target = data['generation']
X_features1 = data.drop(['generation', 'year', 'day', 'area2', 'month'], axis = 1, inplace = False)
X_features2 = data[['month', 'day', 'area2']]

#지역, 날짜 변수는 one-hot encoding, 그 외의 변수는 min-max scaler 사용
#min-max scaler
#발전량, 지역, 날짜 변수 제외 min-max scaling
from sklearn.preprocessing import MinMaxScaler
scaler_ = MinMaxScaler()
scaler_.fit(X_features1)
X_scaled = scaler_.transform(X_features1)
X_features = pd.DataFrame(X_scaled, columns = X_features1.columns)
#날짜, 지역 one-hot encoding
X_features_ohe =pd.get_dummies(X_features2, columns=['month', 'day', 'area2'])
X_features_ = pd.concat([X_features, X_features_ohe], axis = 1)
#발전량 로그 정규화
y_target_log = np.log1p(y_target)
# 7:3으로 훈련, 테스트 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(X_features_, y_target_log, test_size=0.3, random_state=1234)

In [6]:
# RIDGE
ridge_reg = Ridge()
ridge_reg = Ridge(alpha=0.05)
ridge_reg.fit(X_train, y_train)

# LGBM
from sklearn.model_selection import KFold
folds = 5
kf = KFold(n_splits = folds)

lgbm_params = {
    'objective' : 'regression',
    'n_estimators' : 500,
    'num_leaves' : 64,
    'max_depth' : 160,
    'learning_rate' : 0.05,
    'min_child_samples' : 120,
    'subsample' : 0.8,
    'random_seed' : 1234
}
train_X = X_features_
train_Y = y_target_log
models = []
oof = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
  X_train = train_X.iloc[train_index]
  X_valid = train_X.iloc[val_index]
  y_train = train_Y.iloc[train_index]
  y_valid = train_Y.iloc[val_index]

  lgb_train = lgb.Dataset(X_train, y_train)
  lgb_eval = lgb.Dataset(X_valid, y_valid, reference = lgb_train)

  model_lgb = lgb.train(lgbm_params,
                        lgb_train,
                        valid_sets = lgb_eval, 
                        num_boost_round = 100,
                        early_stopping_rounds = 20,
                        verbose_eval = 10,)
  
  y_pred = model_lgb.predict(X_valid, num_iteration = model_lgb.best_iteration)
  models.append(model_lgb)
  oof[val_index] = y_pred

#RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor as rf
train_x = X_features_
train_y = y_target_log

models_rf = []
oof_rf = np.zeros(len(train_x))
for train_index, val_index in kf.split(train_x):
  X_train = train_x.iloc[train_index]
  X_valid = train_x.iloc[val_index]
  y_train = train_y.iloc[train_index]
  y_valid = train_y.iloc[val_index]
  model_rf = rf(
      n_estimators = 50,
      max_features = 6,
      min_samples_leaf = 7,
      random_state = 1234
  )
  model_rf.fit(X_train, y_train)
  y_pred = model_rf.predict(X_valid)  
  models_rf.append(model_rf)
  oof_rf[val_index] = y_pred

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2156
[LightGBM] [Info] Number of data points in the train set: 23572, number of used features: 66
[LightGBM] [Info] Start training from score 5.015234
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l2: 2.89099
[20]	valid_0's l2: 3.88662
Early stopping, best iteration is:
[1]	valid_0's l2: 1.90582
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001694 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2185
[LightGBM] [Info] Number of data points in the train set: 23573, number of used features: 67
[LightGBM] [Info] Start training from score 5.319915
Training until validation scores don't improve for 20 rounds
[10]	valid_0's l2: 1.50116
[20]	valid_0's l2: 1.50326
[30]	valid_0's l2: 

# gui 구현

In [8]:
#설명창 띄우기
def intro(event):
    intro = Tk()
    intro.geometry("950x700")
    intro.option_add("*Font", "나눔고딕 15")
    intro.title("태양광 예측 서비스 -  서비스 소개")  
    intro.resizable(False, False)
    
    photo = PhotoImage(file='4조 프로젝트 발표자료.png', master=intro)
    imageLabel = Label(intro, image = photo)
    imageLabel.pack()  
    intro.resizable(False, False) #크기 조정 못하게 하는 거
    intro.mainloop()

#시작하기 누르면 나오는 메뉴창    
def start(event):
    start = Tk()
    start.geometry("1000x500")
    start.option_add("*Font", "나눔고딕 20")
    start.title("태양광 예측 서비스")
    start.resizable(False, False)
    
    photo = PhotoImage(file='프로젝트_배경1.png', master=start)
    imageLabel = Label(start, image = photo)
    imageLabel.pack() 
    
    #버튼1
    startbt = Button(start, text='우리 지역 평균', command = click_area, width=15, height=4, bg ='white')
    startbt.place(relx=0.15, rely=0.34)
    startbt2 = Button(start, text='태양광 예측 보기', command = click_prediction, width=15, height=4, bg='white')
    startbt2.place(relx=0.55, rely=0.34)
    start.mainloop()
    
 # 우리 지역 평균 누르면 나오는 창  
def click_area():
    global combobox
    global date
    click_area = Tk()
    click_area.geometry("1000x500")
    click_area.option_add("*Font", "나눔고딕 15")
    click_area.title("태양광 예측 서비스 - 우리 지역 평균")
    click_area.resizable(False, False)
    
    photo = PhotoImage(file='프로젝트_배경1.png', master=click_area)
    imageLabel = Label(click_area, image = photo)
    imageLabel.pack()
    
    #라벨들
    label = Label(click_area, text = '<발전량이 궁금한 지역을 입력해주세요>')
    label.place(relx=0.32, rely=0.24)
    label2 = Label(click_area, text = '<예상 발전량이 궁금한 날짜를 입력해주세요>')
    label2.place(relx=0.3, rely=0.48)
    
    #지역 고르는 창
    area = list(df['area'].unique())
    combobox = ttk.Combobox(click_area, height=4, values = area)
    combobox.place(relx=0.35, rely=0.34)
    combobox.set('시/도 선택') 
        
    date = Entry(click_area, width=45)
    date.insert(0,'yyyy-mm-dd')
    date.place(relx=0.23,rely=0.58)   
    
    #선택 버튼을 누르면 위에서 고른 것이 저장됨, 그리고 다음 함수 시작
    def search_result():
        global select_area
        global date_input
        select_area = combobox.get()
        date_input = date.get().split('-')
        areacmd()
        
    btn = Button(click_area, text = '선택', command = search_result)
    btn.place(relx=0.45, rely=0.72)
    click_area.mainloop()
    
def areacmd():
    global result
    areacmd = Tk()
    areacmd.geometry("800x600")
    areacmd.option_add("*Font", "나눔고딕 15")
    areacmd.title("태양광 예측 서비스 - 우리 지역 평균")
    dict = {'강원도':0, '경기도':1, '경상남도':2, '경상북도':3, '광주시':4, '대구시':5, '대전시':6, '부산시':7 ,'서울시':8, '세종시':9, '울산시':10, '인천시':11,'전라남도':12, '전라북도':13, '제주도':14, '충청남도':15, '충청북도':16}
    area2_input = dict[select_area]
    areacmd.resizable(False, False)
    
    photo = PhotoImage(file='프로젝트_배경2.png', master=areacmd)
    imageLabel = Label(areacmd, image = photo)
    imageLabel.pack()
        
    year_input = int(date_input[0])
    month_input = float(date_input[1])
    day_input = float(date_input[2])
        
    # 사용자가 입력한 지역과 월의 평균 발전량
    monthly = data.loc[(data['area2'] == area2_input) & (data['month'] == month_input)]
    avg_mon = monthly['generation'].mean()

    # 사용자가 입력한 지역과 날짜의 예측 발전량
    temp_input = data.loc[(data['area2'] == area2_input) & (data['day'] == day_input) & (data['month'] == month_input)]
    temp_col = ['avgTa', 'avgWs', 'avgRhm', 'ssDur', 'sumSsHr', 'sumGsr','avgTca', 'PM10', 'PM25']
    avg_input = temp_input[temp_col].mean()

    avg = pd.DataFrame(avg_input)
    avg = avg.T

    #입력받은 area, month, day 조합해 df만들기
    w = np.array([area2_input, day_input, month_input])
    w = w.reshape(1,-1)
    weather_input = pd.DataFrame(w, columns = ['area2', 'day', 'month']) 

    #전체 인코딩, 스케일링
    avg_scaled = scaler_.transform(avg)
    avg_feature = pd.DataFrame(avg_scaled, columns = avg.columns)

    weather_df= pd.get_dummies(weather_input, columns = ['month', 'day', 'area2'])
    result = pd.concat([avg_feature, weather_df], axis = 1)
    #feature 수 맞춰주기 위해 X_features_의 행 1개를 가져와서 모두 0으로 입력한 뒤 concat
    cc = X_features_.head(1)
    cc[:] = 0
    result = pd.concat([cc, result], axis = 0, ignore_index = True)
    result = result.fillna(0)
    result[:] = result[:].astype(float)

    #PREDICTION
    #result의 첫째 행은 0으로 이루어진 행이니까 result[1]로 예측값 가져오기
    rid_p = ridge_reg.predict(result)[1] 
    lgb_p = model_lgb.predict(result)[1]
    rf_p = model_rf.predict(result)[1]
    # rmsle수치로 가중치 계산 후 결과값 앙상블
    # rmsle 수치가 작을 수록 성능이 더 좋다는 의미이니까 가중치를 더 크게 부여했음.
    real = rid_p*0.5 + lgb_p*0.36 + rf_p*0.14
    pred_gen = np.expm1(real)
    
    #결과를 라벨로 출력
    
    label0 = Label(areacmd,text='<평균 발전량>',background='orange')
    label0.place(relx=0.35, rely=0.1) 
    
    label1 = Label(areacmd,text='%d월의 평균 태양광 발전량은 %.3f kW입니다.' %(month_input, avg_mon) ,background = 'white')
    label1.place(relx=0.15, rely=0.2)
    
    label0 = Label(areacmd,text='<예측 발전량과 경제적/환경적 효과>',background='skyblue')
    label0.place(relx=0.25, rely=0.35) 
    
    label2 = Label(areacmd,text='%d년 %d월 %d일의 예측 태양광 발전량은 %.3f kW입니다.' %(year_input, month_input, day_input, pred_gen) ,background = 'white')
    label2.place(relx=0.1, rely=0.45)
    
    label3 = Label(areacmd,text='이 발전량은 경유차 %d대분 초미세먼지 저감효과와 같습니다.' %(math.trunc(pred_gen/4.2)) ,background = 'white')
    label3.place(relx=0.11, rely=0.55)
                    
    label4 =  Label(areacmd,text='이 발전량은 소나무 %d그루를 심은 것과 같습니다.' %(math.trunc(pred_gen/0.3*2.5)) ,background = 'white')
    label4.place(relx=0.13, rely=0.65)
                    
    label5 =  Label(areacmd,text='이 발전량은 온실가스 %.3fkg을 감축시킨 것과 같습니다.' %(pred_gen*0.46625) ,background = 'white') 
    label5.place(relx=0.13, rely=0.75)
                    
    label6 =  Label(areacmd,text='이 발전량으로 %d원의 전기요금을 줄일 수 있습니다.' %(pred_gen*80) ,background = 'white')   
    label6.place(relx=0.14, rely=0.85)
    areacmd.mainloop()
        
# 태양광 예측 누르면 나오는 창
def click_prediction():
    click_prediction = Tk()
    click_prediction.geometry("1000x500")
    click_prediction.option_add("*Font", "나눔고딕 15")
    click_prediction.title("태양광 예측 서비스 - 태양광 예측")
    click_prediction.resizable(False, False)
    
    photo = PhotoImage(file='프로젝트_배경1.png', master=click_prediction)
    imageLabel = Label(click_prediction, image = photo)
    imageLabel.pack()
    
    label = Label(click_prediction, text = '<지역을 선택해주세요>')
    label.place(relx=0.4, rely=0.05)
    
    area = list(df['area'].unique())
    combobox = ttk.Combobox(click_prediction, height=4, values = area)
    combobox.place(relx=0.37, rely=0.12)
    combobox.set('시/도 선택')
    
    label = Label(click_prediction, text = '<내일의 온도를 입력해주세요>')
    label.place(relx=0.37, rely=0.25)
    
    ent1 = Entry(click_prediction)
    ent1.place(relx=0.38, rely=0.32)
    
    label = Label(click_prediction, text = '<내일의 10um 미세먼지 농도를 입력해주세요>')
    label.place(relx=0.3, rely=0.45)
    
    ent2 = Entry(click_prediction)
    ent2.place(relx=0.35, rely=0.52, width=300)
    
    label = Label(click_prediction, text = '<내일의 2.5um 미세먼지 농도를 입력해주세요>')
    label.place(relx=0.30, rely=0.65)
    
    ent3 = Entry(click_prediction)
    ent3.place(relx=0.35, rely=0.72, width=300)
    
    #선택 버튼을 누르면 위에서 고른 것이 저장됨, 그리고 다음 함수 시작
    def search_result():
        global select_area2
        global temp
        global pm10
        global pm25
        select_area2 = combobox.get()
        temp = ent1.get()
        pm10 = ent2.get()
        pm25 = ent3.get()
        predcmd()
        
    btn = Button(click_prediction, text = '선택', command = search_result)
    btn.place(relx=0.45, rely=0.8)
    click_prediction.mainloop()
    
def predcmd():
    predcmd = Tk()
    predcmd.geometry("800x600")
    predcmd.option_add("*Font", "나눔고딕 15")
    predcmd.title("태양광 예측 결과")
    
    photo = PhotoImage(file='프로젝트_배경2.png', master=predcmd)
    imageLabel = Label(predcmd, image = photo)
    imageLabel.pack()
    
    from datetime import datetime
    from datetime import timedelta
    from pytz import timezone
    
    KST = timezone('Asia/Seoul')
    now= datetime.now() 
    today = now.astimezone(KST)  
    tomorrow = (today + timedelta(days=1)).strftime("%Y/%m/%d") 
    #사용자가 온도, 미세먼지 2종 데이터 입력
    #시연을 하는 날의 다음날 일기예보 데이터를 입력해 발전량을 보여주는 것!
    
    dict = {'강원도':0, '경기도':1, '경상남도':2, '경상북도':3, '광주시':4, '대구시':5, '대전시':6, '부산시':7 ,'서울시':8, '세종시':9, '울산시':10, '인천시':11,'전라남도':12, '전라북도':13, '제주도':14, '충청남도':15, '충청북도':16}

    area2_input = dict[select_area2]
    temp_input = float(temp)
    pm10_input = float(pm10)
    pm25_input = float(pm25)
    avgTa_insert = pd.DataFrame([temp_input], columns = ['avgTa'])
    dust_insert = pd.DataFrame({'PM10' : [pm10_input], 'PM25' : [pm25_input]})
    
    col = ['avgWs', 'avgRhm', 'ssDur', 'sumSsHr', 'sumGsr','avgTca']
    year_split = int(tomorrow.split('/')[0])
    month_split = float(tomorrow.split('/')[1])
    day_split = float(tomorrow.split('/')[2])
    #tomorrow의 월과 일 추출해 데이터 평균내기
    user_input = data.loc[(data['area2'] == area2_input) & (data['day'] == day_split) & (data['month'] == month_split)]
    avg2 = user_input[col].mean()
    avg2 = pd.DataFrame(avg2)
    avg2 = avg2.T
    avg2 = pd.concat([avgTa_insert, avg2, dust_insert], axis = 1)
    
    #입력받은 area, tomorrow의 month, day 조합해 df만들기
    w2 = np.array([area2_input, day_split, month_split])
    w2 = w2.reshape(1,-1)
    weather2_input = pd.DataFrame(w2, columns = ['area2', 'day', 'month']) 

    #전체 인코딩, 스케일링
    avg2_scaled = scaler_.transform(avg2)
    avg2_feature = pd.DataFrame(avg2_scaled, columns = avg2.columns)

    weather2_df= pd.get_dummies(weather2_input, columns = ['month', 'day', 'area2'])
    result2 = pd.concat([avg2_feature, weather2_df], axis = 1)
    #feature 수 맞춰주기 위해 X_features_의 행 1개를 가져와서 모두 0으로 입력한 뒤 concat
    cc = X_features_.head(1)
    cc[:] = 0
    result2 = pd.concat([cc, result2], axis = 0, ignore_index = True)
    result2 = result2.fillna(0)
    result2[:] = result2[:].astype(float)
    rid_p = ridge_reg.predict(result2)[1] 
    lgb_p = model_lgb.predict(result2)[1]
    rf_p = model_rf.predict(result2)[1]
    real = rid_p*0.5 + lgb_p*0.36 + rf_p*0.14
    pred_gen = np.expm1(real)
    
    
    #결과를 라벨로 출력

    label0 = Label(predcmd,text='<예측 발전량>',background='orange')
    label0.place(relx=0.4, rely=0.1) 
    
    label1 = Label(predcmd,text='%d년 %d월 %d일의 태양광 예측 발전량은 %.3f kW입니다.' %(year_split, month_split, day_split, pred_gen), background = 'white')
    label1.place(relx=0.15, rely=0.2)
    
    label2 = Label(predcmd,text='<경제적/환경적 효과>',background='skyblue')
    label2.place(relx=0.35, rely=0.35) 
    
    label3 = Label(predcmd,text='이 발전량은 경유차 %d대분 초미세먼지 저감효과와 같습니다.' %(math.trunc(pred_gen/4.2)),background = 'white')
    label3.place(relx=0.11, rely=0.45)
                    
    label4 =  Label(predcmd,text='이 발전량은 소나무 %d그루를 심은 것과 같습니다.' %(math.trunc(pred_gen/0.3*2.5)), background = 'white')
    label4.place(relx=0.13, rely=0.55)
                    
    label5 =  Label(predcmd,text='이 발전량은 온실가스 %.3fkg을 감축시킨 것과 같습니다.' %(pred_gen*0.46625), background = 'white')
    label5.place(relx=0.13, rely=0.65)
                    
    label6 =  Label(predcmd,text='이 발전량으로 %d원의 전기요금을 줄일 수 있습니다.' %(pred_gen*80), background = 'white')   
    label6.place(relx=0.14, rely=0.75)  
    predcmd.mainloop()
        
    
# 시작 페이지
win = Tk()
win.geometry("1000x500")
win.option_add("*Font", "나눔고딕 15")
win.title("태양광 예측 서비스")

photo = PhotoImage(file='프로젝트_메인화면.png', master=win)
imageLabel = Label(win, image = photo)
imageLabel.pack()  

stbt = Button(win)
stbt.config(width=15, height=4)
stbt.config(text='서비스 소개', bg = 'white')
stbt.bind("<Button-1>", intro) 
stbt.place(relx=0.55, rely=0.44)

stbt = Button(win, text='시작하기', bg = 'white')
stbt.config(width=15, height=4)
stbt.config(text='시작하기')
stbt.bind("<Button-1>", start) 
stbt.place(relx=0.25, rely=0.44)
win.resizable(False, False)

win.mainloop()